In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, LSTM, Subtract, Concatenate
from tensorflow.keras.models import Model
from Environment import *
from tqdm import tqdm
import matplotlib.pyplot as plt
from copy import deepcopy
import os 
import numpy as np

In [2]:
#User Agent Actor
input_A = Input(shape = 4) #Input of user actor
input_B = Input(shape = (4, 6)) #Input of assistant actor/ Output of user actor 
input_C = Input(shape = (11,11,3)) #Icon layout/ input of assistant actor
input_D = Input(shape = 4) #Output of assistant actor

x = Subtract()([input_A[:, 2:], input_A[:, :2]])
x = Dense(32, activation = 'relu')(x)
x = Dense(64, activation = 'relu')(x)
x = Dense(4, activation = 'softmax')(x)

user_actor = Model(input_A, x)
user_actor.summary()

#Assistant Agent Actor
a = Dense(32, activation = 'relu')(input_B)
a = LSTM(32, activation = 'tanh')(a)

b = tf.keras.layers.Conv2D(filters = 2, kernel_size = 3, activation = 'relu')(input_C)
b = tf.keras.layers.MaxPooling2D()(b)
b = tf.keras.layers.Flatten()(b)
b = tf.keras.layers.Dense(32, activation = 'relu')(b)

a = Concatenate()([a, b])
a = Dense(32, activation = 'relu')(a)
a = Dense(4, activation = 'softmax')(a)

asst_actor = Model(inputs = [input_B, input_C], outputs = a)
asst_actor.summary()

#Centralized Critic layers
x = Subtract()([input_A[:, 2:], input_A[:, :2]])
x = Dense(32, activation = 'relu')(x)
x = Dense(64, activation = 'relu')(x)
x = Dense(4, activation = 'softmax')(x)

y = Dense(32, activation = 'relu')(input_B)
y = LSTM(32, activation = 'tanh')(y)

z = tf.keras.layers.Conv2D(filters = 2, kernel_size = 3, activation = 'relu')(input_C)
z = tf.keras.layers.MaxPooling2D()(z)
z = tf.keras.layers.Flatten()(z)
z = tf.keras.layers.Dense(32, activation = 'relu')(z)

w = Dense(32, activation = 'relu')(input_D)

y = Concatenate()([x, y, z, w])
y = Dense(32, activation = 'relu')(y)
y = Dense(1)(y)

cent_critic = Model(inputs = [input_A, input_B, input_C, input_D], outputs = y)
cent_critic.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [(None, 2)]          0           input_1[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_1 (Te [(None, 2)]          0           input_1[0][0]                    
__________________________________________________________________________________________________
subtract (Subtract)             (None, 2)            0           tf_op_layer_strided_slice[0][0]  
                                                                 tf_op_layer_strided_sl